# Code from part 1 - please scroll to Part 2

In [ ]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import urllib # library to handle requests
import requests
# import BeautifulSoup
from bs4 import BeautifulSoup
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
import lxml
import csv

In [25]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

#get response from url
res = requests.get(url).text

#scrape
soup = BeautifulSoup(res, 'lxml')
table = soup.find('table')

#csv
csv_file = open('Toronto.csv', 'w')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Postal Code','Borough','Neighborhood'])

rowdata = ''
xx = ''
for record in table.find_all('tr'):
    rowdata = ''
    for data in record.find_all('td'):
        rowdata = rowdata + ',' + data.text
    xx = rowdata[1:-1]
    xx = xx.split(',')

    if len(xx) > 1:
       postcode = xx[0]
       borough = xx[1] 
       neighborhood = xx[2]+', '
       if neighborhood == 'Not assigned, ':
          neighborhood = borough + ', '
       if borough != 'Not assigned':
          csv_writer.writerow([postcode,borough,neighborhood])
csv_file.close()

# data wrangling

df = pd.read_csv('Toronto.csv')
df2 = df[['Postal Code','Borough']]
df3 = df[['Postal Code','Neighborhood']]
df2 = df2.drop_duplicates(subset=None, keep='first', inplace=False)
df3 = df3.groupby(['Postal Code'], as_index = False).sum()
for i in range(df3.shape[0]):
    xx = df3['Neighborhood'][i]
    xx = xx[0:-2]
    df3['Neighborhood'][i] = xx
df = pd.merge(df2, df3, on='Postal Code')

# Part 2

In [ ]:
Now that you have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

Given that this package can be very unreliable, in case you are not able to get the geographical coordinates of the neighborhoods using the Geocoder package, here is a link to a csv file that has the geographical coordinates of each postal code: http://cocl.us/Geospatial_data

Use the Geocoder package or the csv file to create the following dataframe:

In [22]:
df_ll= pd.read_csv("http://cocl.us/Geospatial_data")
df_merge = pd.merge(df, df_ll, on='Postal Code')

In [27]:
# keep only boroughs containing the word Toronto 
toronto_data = df_merge[df_merge['Borough'].str.contains('Toronto')].reset_index(drop=True)

print('The dataframe has {} boroughs and {} postcodes. \n'.format(
        len(toronto_data['Borough'].unique()),
        toronto_data.shape[0]
    )
)
toronto_data.head(15)

The dataframe has 4 boroughs and 38 postcodes. 



,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
5,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
6,M6G,Downtown Toronto,Christie,43.669542,-79.422564
7,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
8,M6H,West Toronto,"Dovercourt Village, Dufferin",43.669005,-79.442259
9,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752
